In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import re
from tqdm import tqdm


In [3]:
os.listdir('pages')

['page18.htm',
 'page24.htm',
 'page30.htm',
 'page31.htm',
 'page25.htm',
 'page19.htm',
 'page9.htm',
 'page33.htm',
 'page27.htm',
 'page26.htm',
 'page32.htm',
 'page8.htm',
 'page22.htm',
 'page23.htm',
 'page21.htm',
 'page34.htm',
 'page20.htm',
 'page3.htm',
 'page11.htm',
 'page10.htm',
 'page2.htm',
 'page0.htm',
 'page12.htm',
 'page13.htm',
 'page1.htm',
 'page5.htm',
 'page17.htm',
 'page16.htm',
 'page4.htm',
 'page6.htm',
 'page14.htm',
 'page28.htm',
 'page29.htm',
 'page15.htm',
 'page7.htm',
 'page.htm']

In [9]:
lst = os.listdir('pages/')

In [15]:
file = open('pages/' + lst[0], 'r')
text = file.read()
h = BeautifulSoup(text, 'html.parser')

In [14]:
len(h.find_all('tr', {'class':'ng-scope'}))

10

In [41]:
zeilen = h.find_all('tr', {'class':'ng-scope'})

In [42]:
zeilen[0]

<tr class="ng-scope" ng-repeat="row in $data"><!-- ngRepeat: col in $columns --><!-- ngIf: col.show() --><td class="ng-scope" data-title-text="business name" filter="$listCtrl.filters[col.name]" ng-if="col.show()" ng-repeat="col in $columns"><!-- ngIf: col.name === 'company' --><div class="company-name ng-scope" ng-attr-title="{{row.name}}" ng-if="col.name === 'company'" title="Bäckerei zur Kettenbrücke M. Leutwiler"><a class="ng-binding" href="/en/search/entity/list/firm/901417?name=b%C3%A4ckerei&amp;searchType=exact" ng-bind-html="highlight(row.name, $listCtrl.tableParams.filter(), 'name')" ng-class="{ 'strike-thru': row.status === 'GELOESCHT'}" ui-sref=".details({ ehraid: row.ehraid })">Bäckerei zur Kettenbrücke M. Leutwiler</a></div><!-- end ngIf: col.name === 'company' --><!-- ngIf: col.name === 'legalSeat' --><!-- ngIf: col.getValue --></td><!-- end ngIf: col.show() --><!-- end ngRepeat: col in $columns --><!-- ngIf: col.show() --><td class="ng-scope" data-title-text="excerpt" fi

In [43]:
zeilen[0].find_all("div")[0].text

'Bäckerei zur Kettenbrücke M. Leutwiler'

In [44]:
h.find_all('tr', {'class':'ng-scope'})[0].find_next('div').find_next('div').find('a')['href']

'https://ag.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-107.083.298&lang=de'

In [45]:
h.find_all('tr', {'class':'ng-scope'})[0].find_next('div') \
    .find_next('div').find_next('div').find_next('div').find_next('div').text

'Aarau'

In [46]:
h.find_all('tr', {'class':'ng-scope'})[0].find_next('div') \
    .find_next('div').find_next('div').find_next('div') \
    .find_next('div').find_next('div').text

'AG - Aargau'

In [48]:
soup_lst = h.find_all('tr', {'class':'ng-scope'})

In [58]:
blist = []
zeilen = h.find_all('tr', {'class':'ng-scope'})
for bäck in soup_lst:
    
    name = bäck.find('div')['title']
    url = bäck.find_next('div').find_next('div').find('a')['href']
    gem = bäck.find_next('div') \
        .find_next('div').find_next('div').find_next('div').find_next('div').text
    kt = bäck.find_next('div') \
        .find_next('div').find_next('div').find_next('div') \
        .find_next('div').find_next('div').text
    
    minidict = {'Bäckerei': name,
                'URL': url,
                'Gemeinde': gem,
                'Kanton': kt}
    
    blist.append(minidict)

In [59]:
df = pd.DataFrame(blist)

In [60]:
df

,Bäckerei,Gemeinde,Kanton,URL
0,Bäckerei zur Kettenbrücke M. Leutwiler,Aarau,AG - Aargau,https://ag.chregister.ch/cr-portal/auszug/zefi...
1,"Bäckerei, Konditorei Fahrni",Dotzigen,BE - Bern,https://be.chregister.ch/cr-portal/auszug/zefi...
2,"Bäckerei, Konditorei Winter, Ilic Milovan",Adliswil,ZH - Zürich,https://zh.chregister.ch/cr-portal/auszug/zefi...
3,"Bäckerei, Konditorei, Café ""Zum Steinenen Trau...",Stein am Rhein,SH - Schaffhausen,https://sh.chregister.ch/cr-portal/auszug/zefi...
4,"Bäckerei, Konditorei, Confiserie Mock",Basel,BS - Basel-Stadt,https://bs.chregister.ch/cr-portal/auszug/zefi...
5,"Bäckerei, Konditorei, Lebensmittel E. Fahrni",Sigriswil,BE - Bern,https://be.chregister.ch/cr-portal/auszug/zefi...
6,"Bäckerei, Tea-Room und Garni Imseng",Saas-Fee,VS - Brig (Oberwallis),https://vo.chregister.ch/cr-portal/auszug/zefi...
7,Bäckerei-Café Betschart,Bonstetten,ZH - Zürich,https://zh.chregister.ch/cr-portal/auszug/zefi...
8,Bäckerei-Café Kohler AG,Hunzenschwil,AG - Aargau,https://ag.chregister.ch/cr-portal/auszug/zefi...
9,Bäckerei-Café Meyer AG,Hitzkirch,LU - Luzern,https://lu.chregister.ch/cr-portal/auszug/zefi...


In [61]:
def get_bäcks(html_code):
    
    soup_lst = html_code.find_all('tr', {'class':'ng-scope'})
    
    blist = []
    for bäck in soup_lst:
    
        name = bäck.find('div')['title']
        url = bäck.find_next('div').find_next('div').find('a')['href']
        gem = bäck.find_next('div') \
            .find_next('div').find_next('div').find_next('div').find_next('div').text
        kt = bäck.find_next('div') \
            .find_next('div').find_next('div').find_next('div') \
            .find_next('div').find_next('div').text
    
        minidict = {'Bäckerei': name,
                'URL': url,
                'Gemeinde': gem,
                'Kanton': kt}
    
        blist.append(minidict)
    
    return blist

In [62]:
get_bäcks(h)

[{'Bäckerei': 'Bäckerei zur Kettenbrücke M. Leutwiler',
  'URL': 'https://ag.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-107.083.298&lang=de',
  'Gemeinde': 'Aarau',
  'Kanton': 'AG - Aargau'},
 {'Bäckerei': 'Bäckerei, Konditorei Fahrni',
  'URL': 'https://be.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-107.180.240&lang=de',
  'Gemeinde': 'Dotzigen',
  'Kanton': 'BE - Bern'},
 {'Bäckerei': 'Bäckerei, Konditorei Winter, Ilic Milovan',
  'URL': 'https://zh.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-189.036.773&lang=de',
  'Gemeinde': 'Adliswil',
  'Kanton': 'ZH - Zürich'},
 {'Bäckerei': 'Bäckerei, Konditorei, Café "Zum Steinenen Trauben" Walz',
  'URL': 'https://sh.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-106.833.950&lang=de',
  'Gemeinde': 'Stein am Rhein',
  'Kanton': 'SH - Schaffhausen'},
 {'Bäckerei': 'Bäckerei, Konditorei, Confiserie Mock',
  'URL': 'https://bs.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-110.597.445&lang=de',
  'Gemeinde': 'Base

In [65]:
lst

['page18.htm',
 'page24.htm',
 'page30.htm',
 'page31.htm',
 'page25.htm',
 'page19.htm',
 'page9.htm',
 'page33.htm',
 'page27.htm',
 'page26.htm',
 'page32.htm',
 'page8.htm',
 'page22.htm',
 'page23.htm',
 'page21.htm',
 'page34.htm',
 'page20.htm',
 'page3.htm',
 'page11.htm',
 'page10.htm',
 'page2.htm',
 'page0.htm',
 'page12.htm',
 'page13.htm',
 'page1.htm',
 'page5.htm',
 'page17.htm',
 'page16.htm',
 'page4.htm',
 'page6.htm',
 'page14.htm',
 'page28.htm',
 'page29.htm',
 'page15.htm',
 'page7.htm',
 'page.htm']

In [68]:
finallist = []

for filename in lst:
    
    file = open('pages/' + filename, 'r')
    text = file.read()
    h = BeautifulSoup(text, 'html.parser')
    
    finallist = finallist + get_bäcks(h)

In [73]:
pd.DataFrame(finallist)

,Bäckerei,Gemeinde,Kanton,URL
0,Bäckerei zur Kettenbrücke M. Leutwiler,Aarau,AG - Aargau,https://ag.chregister.ch/cr-portal/auszug/zefi...
1,"Bäckerei, Konditorei Fahrni",Dotzigen,BE - Bern,https://be.chregister.ch/cr-portal/auszug/zefi...
2,"Bäckerei, Konditorei Winter, Ilic Milovan",Adliswil,ZH - Zürich,https://zh.chregister.ch/cr-portal/auszug/zefi...
3,"Bäckerei, Konditorei, Café ""Zum Steinenen Trau...",Stein am Rhein,SH - Schaffhausen,https://sh.chregister.ch/cr-portal/auszug/zefi...
4,"Bäckerei, Konditorei, Confiserie Mock",Basel,BS - Basel-Stadt,https://bs.chregister.ch/cr-portal/auszug/zefi...
5,"Bäckerei, Konditorei, Lebensmittel E. Fahrni",Sigriswil,BE - Bern,https://be.chregister.ch/cr-portal/auszug/zefi...
6,"Bäckerei, Tea-Room und Garni Imseng",Saas-Fee,VS - Brig (Oberwallis),https://vo.chregister.ch/cr-portal/auszug/zefi...
7,Bäckerei-Café Betschart,Bonstetten,ZH - Zürich,https://zh.chregister.ch/cr-portal/auszug/zefi...
8,Bäckerei-Café Kohler AG,Hunzenschwil,AG - Aargau,https://ag.chregister.ch/cr-portal/auszug/zefi...
9,Bäckerei-Café Meyer AG,Hitzkirch,LU - Luzern,https://lu.chregister.ch/cr-portal/auszug/zefi...


In [75]:
df['Kanton'].value_counts()

BE - Bern                 2
AG - Aargau               2
ZH - Zürich               2
LU - Luzern               1
VS - Brig (Oberwallis)    1
BS - Basel-Stadt          1
SH - Schaffhausen         1
Name: Kanton, dtype: int64